In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.stats as ss
import math as math

import sklearn.model_selection as ms 

In [2]:
# load your data (don't touch, just run)
"""
data = []
f = open('covtype.data','r')
while(1):
    line = f.readline()
    if  len(line) < 100:
        print line
    
    if len(line) == 0: break
    data.append(np.array([float(k) for k in line.split(',')]))
    if len(data) % 100000 == 0:
        print len(data)
        
f.close
data = np.vstack(data)
N = data.shape[0]
idx = np.random.permutation(N)


X_test = data[:N/5,:]
X_train = data[N/5:,:]
y_test = X_test[:,-1]
y_train = X_train[:,-1]
X_test = X_test[:,:-1]
X_train = X_train[:,:-1]


sio.savemat('covtype.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})

data = sio.loadmat('covtype.mat')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train'][0]
y_test = data['y_test'][0]

y_idx_train = [np.where(np.equal(y_train,k))[0] for k in np.unique(y_train)]

for i in xrange(len(y_idx_train)):
    y_idx = y_idx_train[i]
    y_idx_train[i] = y_idx[np.random.choice(len(y_idx),len(y_idx)/1000+1,replace=False)]
    
y_idx_train = np.hstack(y_idx_train)
y_idx_train = np.random.permutation(y_idx_train)

X_train = X_train[y_idx_train,:]
y_train = y_train[y_idx_train]

sio.savemat('covtype_reduced.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})


"""

"\ndata = []\nf = open('covtype.data','r')\nwhile(1):\n    line = f.readline()\n    if  len(line) < 100:\n        print line\n    \n    if len(line) == 0: break\n    data.append(np.array([float(k) for k in line.split(',')]))\n    if len(data) % 100000 == 0:\n        print len(data)\n        \nf.close\ndata = np.vstack(data)\nN = data.shape[0]\nidx = np.random.permutation(N)\n\n\nX_test = data[:N/5,:]\nX_train = data[N/5:,:]\ny_test = X_test[:,-1]\ny_train = X_train[:,-1]\nX_test = X_test[:,:-1]\nX_train = X_train[:,:-1]\n\n\nsio.savemat('covtype.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})\n\ndata = sio.loadmat('covtype.mat')\nX_train = data['X_train']\nX_test = data['X_test']\ny_train = data['y_train'][0]\ny_test = data['y_test'][0]\n\ny_idx_train = [np.where(np.equal(y_train,k))[0] for k in np.unique(y_train)]\n\nfor i in xrange(len(y_idx_train)):\n    y_idx = y_idx_train[i]\n    y_idx_train[i] = y_idx[np.random.choice(len(y_idx),len(y_idx)/1000+1,repla

In [3]:

data = sio.loadmat('covtype_reduced.mat')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train'][0]
y_test = data['y_test'][0]

print(np.unique(y_train), np.unique(y_test))

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print()


[1. 2. 3. 4. 5. 6. 7.] [1. 2. 3. 4. 5. 6. 7.]
(468, 54) (116202, 54) (468,) (116202,)



In [4]:
def entropy(label):
    hash_map = {}
    array = []
    total = 0

    for i in range(len(label)):
        x = label[i]
        if x not in hash_map:
            hash_map[x] = 1
            array.append(x)
        else:
            hash_map[x] = hash_map[x] + 1
        total = total + 1
    
    entropy = 0
    for key in hash_map:
        entropy = entropy + (-1 * (hash_map[key] / total) * math.log2(hash_map[key] / total))
    return entropy
 
def cond_entropy(label,split):
    main_dict = {}
    split_dict = {}
    total = 0
    
    for i in range(len(split)):
        x = split[i]
        y = label[i]
        if x not in main_dict:
            main_dict[x] = {}
            split_dict[x] = 0
        
        hash_map = main_dict[x]
        split_dict[x] = split_dict[x] + 1
        
        if y not in hash_map:
            hash_map[y] = 0
        hash_map[y] = hash_map[y] + 1
        total += 1
    
    entropy = 0
    for key1 in main_dict:
        hash_map = main_dict[key1]
        localTotal = split_dict[key1]
        
        for key2 in hash_map:
            part1 = hash_map[key2] / total
            part2 = hash_map[key2] / localTotal
            if part2 == 0:
                entropy += 0
            else:
                entropy += (-1 * part1 * math.log2(part2))
    
    return entropy
        
    
        

random_sequences = sio.loadmat('random_sequences.mat')

s1 = random_sequences['s1'][0]
s2 = random_sequences['s2'][0]

print('entropy = ', entropy(s1))
print('conditional entropy = ', cond_entropy(s1,s2))

entropy =  3.3141823231610834
conditional entropy =  3.302959881613517


In [25]:
def find_best_split(x,y):
    print(len(x))
    print
    best_feat = 0
    splitval = 0.
    set1 = []
    set2 = []
    
    entropyVal = entropy(y)
    max_value = 0
    
    
    for i in range(len(x[0])):
        x_values = []
        minX = 99999999999
        maxX = 0
        for j in range(len(x)):
            val = x[j][i]
            x_values.append(x[j][i])
            if val < minX:
                minX = val
            if val > maxX:
                maxX = val
        
        r = round(maxX - minX)
        for index in range(r):
            k = index + minX;
            newX_values = []
            for j in range(len(x_values)):
                if x_values[j] < k:
                    newX_values.append(0)
                else:
                    newX_values.append(1)

            cond_entropy_temp = cond_entropy(y, newX_values)
            possible_result = entropyVal - cond_entropy_temp
            if possible_result > max_value:
                max_value = possible_result
                best_feat = i
                splitval = k
    
    for i in range(len(x)):
        if (x[i][best_feat] < splitval):
            set1.append(i)
        else:
            set2.append(i)
    
    return best_feat, splitval, set1, set2


best_feat, splitval, set1, set2 = find_best_split(X_train, y_train)

y_new = y_train * 0
y_new[set2] = 1

print(best_feat,splitval)


print('information gained in first step', entropy(y_train) - cond_entropy(y_train,y_new))

468
0 2844.0
information gained in first step 0.34265604135138417


In [11]:
def purity(y):
    return ss.mode(y)[1]/len(y+0.)
    


class Node:
    def __init__(self,  sample_idx, nodeid,  is_leaf = True):
        self.is_leaf = is_leaf
        self.id = nodeid
        self.sample_idx = sample_idx
        self.children = []
        
        
        
    def visit_node(self, x):
        if self.is_leaf:
            return self.label
        
        
        return self.children[0].visit_node(x)
        
    def add_split_details(self, splitfeat, splitval)  :
        self.splitfeat = splitfeat
        self.splitval = splitval
    
        
class Tree:
    def __init__(self, x,y):
        m = len(y)
        self.x = x
        self.y = y
        self.maxid = -1
        self.root = self.construct_node(np.array(range(m)))
        self.leaves = [self.root]
        
    def print_tree(self):
        print('printing tree...')
        def print_node(parent, node):
            print(node.id, end='')
            
            if parent is not None:
                print(', parent ', parent.id,end='')
            else:
                print(', ROOT',end='')
                
            print(', label ', node.label, end='')
            if node.is_leaf: 
                print(', LEAF, ', 'nsamples %d, purity %.2f' %(len(node.sample_idx), purity(self.y[node.sample_idx])))
            else:
                print(', NONLEAF, split %d, val %.2f' % (node.splitfeat, node.splitval))
            if not node.is_leaf:
                for ch in node.children:
                    print_node(node, ch)
        print_node(None, self.root)
        
        
    def construct_node(self, sample_idx):
        node = Node(sample_idx, self.maxid + 1,  True)
        node.label = 0 # fill me in
        node.entropy = entropy(self.y[sample_idx])
        node.num_mistakes = np.sum(np.not_equal(node.label, self.y[sample_idx]))
        self.maxid += 1
        return node
        


    def report_train_err(self):
        total_mistakes = 0
        for leaf in self.leaves:
            total_mistakes += leaf.num_mistakes
        return total_mistakes / (len(self.y)+0.)
        
    
    
    def predict(self,x):
        return self.root.visit_node(x)


    

In [41]:
def get_test_err(tree):
    # get test error
    num_test_mistakes = 0
    for k in range(len(y_test)):
        x,y = X_test[k,:],y_test[k]
        if y != tree.predict(x):
            num_test_mistakes += 1
    return num_test_mistakes / (len(y_test)+0.)



tree = Tree(X_train,y_train)
## tree.print_tree()
## print('current train err:', tree.report_train_err())
## print('current test err:', get_test_err(tree))


# my first split
best_feat, splitval, set1, set2 = find_best_split(X_train, y_train)
 
left_child = tree.construct_node(set1)
right_child = tree.construct_node(set2)
tree.root.is_leaf = False
tree.leaves.pop(tree.leaves.index(tree.root))
tree.root.add_split_details(splitfeat = best_feat, splitval = splitval)


tree.root.children = [left_child, right_child]
tree.leaves.extend(tree.root.children)
tree.print_tree()

## print('one step train err:', ni)
print('one step test err:', get_test_err(tree))





printing tree...
0, ROOT, label  0, LEAF,  nsamples 468, purity 0.44
current train err: 1.0
current test err: 1.0
468
printing tree...
0, ROOT, label  0, NONLEAF, split 0, val 2844.00
1, parent  0, label  0, LEAF,  nsamples 116, purity 0.46
2, parent  0, label  0, LEAF,  nsamples 352, purity 0.51
one step test err: 1.0


In [44]:
'two step test error'

best_feat, splitval, set1, set2 = find_best_split(X_train, y_train)

left_child = tree.construct_node(set1)
right_child = tree.construct_node(set2)
tree.root.is_leaf = False

tree.root.add_split_details(splitfeat = best_feat, splitval = splitval)


X_train2 = []
y_train2 = []

for i in range(len(X_train)):
    val = X_train[i][best_feat]
    if val < splitval:
        x = X_train[i]
        x2 = np.delete(x, best_feat)
        X_train2.append(x2)
        y_train2.append(y_train[i])

print(len(set1))
print(len(X_train2))
print(len(X_train2[0]))
print(len(y_train2))

X_train2 = np.asarray(X_train2)
y_train2 = np.asarray(y_train2)
tree.print_tree()





468
116
116
53
116
printing tree...
0, ROOT, label  0, NONLEAF, split 0, val 2844.00
1, parent  0, label  0, LEAF,  nsamples 116, purity 0.46
2, parent  0, label  0, LEAF,  nsamples 352, purity 0.51


In [45]:
best_feat, splitval, set1, set2 = find_best_split(X_train2, y_train2)

left_tree = Tree(X_train2,y_train2)

left_child2 = left_tree.construct_node(set1)
right_child2 = left_tree.construct_node(set2)
left_tree.root.is_leaf = False
left_tree.root.add_split_details(splitfeat = best_feat, splitval = splitval)
left_tree.root.children = [left_child2, right_child2]

left_tree.print_tree()

116
printing tree...
0, ROOT, label  0, NONLEAF, split 5, val 242.00
1, parent  0, label  0, LEAF,  nsamples 101, purity 0.50
2, parent  0, label  0, LEAF,  nsamples 15, purity 0.47
